In [35]:
import os
import numpy as np
import gradio as gr
import joblib
import librosa
import soundfile
import wave
from collections import Counter

In [36]:
model = joblib.load('./SEP.h5')
def extract_feature(audio_file):
    with soundfile.SoundFile(audio_file) as sound_file:
        X = sound_file.read(dtype="float32")
        result = np.array([])
        sample_rate = sound_file.samplerate
        mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
        result = np.hstack((result,mfccs))
        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
        result = np.hstack((result,chroma))
        mel = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0)
        result = np.hstack((result,mel))
        result = result.reshape(1, -1)
        y_pred = model.predict(result)
        print(y_pred)
        return y_pred


In [47]:
def main():
    audio_input = gr.Audio(sources=["upload"],type="filepath")
    output_text = gr.Textbox()
    iface = gr.Interface(fn=extract_feature,
                         inputs=audio_input,
                         outputs=output_text,
                         title="SEP",
                         description="upload an audio file and submit",
                         examples=[['calm.wav'],['sad.wav'],['happy.wav'],['angry.wav']])
    iface.launch()


In [48]:
if __name__=="__main__":
    main()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


['angry']
['calm']
